In [20]:
import os
import numpy as np
import pandas as pd
import random

In [133]:
class AssetOverlap():
    def __init__(self):
        self.readFunds()
        self.readSP500()

        
    def readFunds(self):
        fundDir = os.listdir("../input/funds")
        funds = pd.DataFrame(columns=["Ticker", "ID"])
        for name in fundDir:
            fund = pd.read_csv("../input/funds/"+name, index_col=0)
            name = name.split(".csv")[0]
            
            fund = fund.rename(columns={fund.columns[-1]:"Ticker", "ID.WEIGHTS":name})
            fund = fund[["ID","Ticker",name]]
            fund = fund[(fund.Ticker.str[:4]!="#N/A") & (fund[name]>0)] #no puts so neg percent and #N/A Unclassified: Unable to parse request at ...  and # N/A Invalid Security
            fund = fund.dropna(axis=0, how='all')
            
            fund[name] = fund[name]/(fund[name].sum())*100 # some have over 100 percent
            
            fund.Ticker = fund.Ticker.fillna("NULL TICKER"+fund.ID)
            
            fund = fund.groupby(["Ticker","ID"]).agg({name:"sum"})# you have to groupby id and ticker or else it may merge multipletimes
            
            funds = funds.merge(fund, how='outer', on=["Ticker","ID"])
        
        
        params = {fundName:"sum" for fundName in self.getFundNames(funds)}
        params.update({"ID":lambda IDS: IDS.iloc[np.argmax([np.all([not char.isdigit() for char in ID]) for ID in IDS])] }) #first id that is all characters or first id
        funds = funds.groupby("Ticker").agg(params)
        funds = funds.reset_index()

        #print(funds.Ticker.duplicated().sum())
        
        funds = funds.fillna(0)
        self.funds = funds

    def getFundMatrix(self):
        return self.funds.loc[:,~self.funds.columns.isin(["ID","Ticker"])]
    
    def readSP500(self):
        self.sp500 = pd.read_csv("../input/s&p500.csv")
        self.sp500 = self.sp500.iloc[:,:3]
        self.sp500 = self.sp500.rename(columns={"Symbol":"Ticker", "Weight":"S&P500"})
        
    def getFundNames(self,funds):
        return [x for x in funds.columns if x not in ["ID", "Ticker"]]
    
    def makePortfolio(self,weights):
        #self.weights = self.weights.loc[self.getFundNames(self.funds)]
        columns = ["Ticker","ID"] + list(weights.index)
        self.funds = self.funds[columns]

        Portfolio = self.getFundMatrix().dot(weights)
        Portfolio = Portfolio.rename({Portfolio.columns[0]:"Portfolio"},axis=1)
        Portfolio[["Ticker","ID"]] = self.funds[["Ticker","ID"]]
        
        Portfolio = pd.merge(Portfolio, self.sp500, how='outer', on="Ticker")
        Portfolio.ID = Portfolio.ID.fillna(Portfolio.Company)
        Portfolio = Portfolio.drop("Company", axis=1)
        return Portfolio
    
    def makeAddStocksPortfolio(self,Portfolio):
        dif = Portfolio["S&P500"]-Portfolio["Portfolio"] # determine which sp500 assets have more than in the portfolio
        self.extra = Portfolio
        self.extra["Dif"] = dif
        self.extra = self.extra[self.extra.Dif>0][["Ticker","ID","Dif"]]
        self.extra = self.extra.rename(columns={"Dif":"Portfolio"})
        AddStocksPortfolio = pd.concat([Portfolio[["Ticker","ID","Portfolio"]],self.extra],axis=0)
        return AddStocksPortfolio
    
    def simpleOverlap(self,weights):
        Portfolio = self.makePortfolio(weights)
        Portfolio[["Portfolio","S&P500"]] = Portfolio[["Portfolio","S&P500"]].fillna(0)
        overlap = np.minimum(np.array(Portfolio["Portfolio"]),np.array(Portfolio["S&P500"]))
        overlap = overlap.sum()
        total = Portfolio.Portfolio.sum()
        return overlap/total*100
    
    def addStocksOverlap(self,weights):
        Portfolio = self.makePortfolio(weights)
        AddStocksPortfolio = self.makeAddStocksPortfolio(Portfolio)
        return self.sp500["S&P500"].sum()/AddStocksPortfolio.Portfolio.sum()*100
    def addStocksPercentCEF(self,weights):
        Portfolio = self.makePortfolio(weights)
        AddStocksPortfolio = self.makeAddStocksPortfolio(Portfolio)
        return Portfolio.Portfolio.sum()/AddStocksPortfolio.Portfolio.sum()*100
        
    def evaluateAssetOverlap(self,weights):
        return self.simpleOverlap(weights)
        
        
        
        

In [139]:
class Discount:
    def __init__(self):
        self.readDiscount()
    def readDiscount(self):
        self.discount = pd.read_csv("../input/Discounts.csv")
        self.discount[["Discount", "52W Discount","Effective","Distribution"]] = self.discount.apply(lambda x: [x["Discount"].strip("%"),x["52W Discount"].strip("%"), x["Effective"].strip("%"), x["Distribution"].strip("%")], axis=1, result_type='expand')
        self.discount = self.discount.replace("--",0)
        self.discount.iloc[:,1:] = self.discount.iloc[:,1:].astype("float")
        
        longterm = self.discount["52W Discount"]-self.discount["Discount"] #max: a pos 52 week is pos, a neg discount is more pos
        longterm = longterm/np.max(longterm) *100# between -inf and 1
        self.discount["longterm"] = longterm
        current = -self.discount["Discount"] # max: a neg discount is pos
        current = current/np.max(-self.discount["Discount"]) * 100 #between -inf and 1
        self.discount["value"] = np.sum([longterm,2*current],axis=0)/3
        self.discount = self.discount.set_index("Ticker")
    
    def getWeightedDiscount(self,weights):
        percentCEF = self.addStocksPercentCEF(weights)/100
        self.discount = self.discount.loc[weights.index]
        weights = np.array(weights).reshape(1,len(weights))
        values = np.array(self.discount.Discount).reshape(len(self.discount),1)
        return percentCEF * np.matmul(weights,values)[0][0]
    
    def evaluateDiscount(self,weights):
        self.discount = self.discount.loc[weights.index]
        weights = np.array(weights).reshape(1,len(weights))
        values = np.array(self.discount.value).reshape(len(self.discount),1)
        
        return np.dot(weights,values)[0][0]
        
          

In [140]:
class TabuSet:
    def inTabuSet(self,option):
        fund1, fund2 = option
        tabuList = [adjustment.getFunds() for adjustment in self.TabuSet[:self.numTabu]]
        return [fund1,fund2] in tabuList or [fund2,fund1] in tabuList

In [141]:
class Adjustment():
    def __init__(self,fund1,fund2,value,weights):
        self.fund1 = fund1
        self.fund2 = fund2
        self.value = value
        self.weights = weights
    def getValue(self):
        return self.value
    def getWeights(self):
        return self.weights
    def getFunds(self):
        return [self.fund1, self.fund2]

In [164]:
class runModel(AssetOverlap, Discount,TabuSet):
    def __init__(self):
        self.readFunds()
        self.readSP500()
        self.readDiscount()
        
        self.numOptions = 10
        self.numTabu = 0
        
        N = len(self.getFundNames(self.funds))
        weights = np.random.uniform(0,1.0,(N,1))
        weights = np.divide(weights,np.sum(weights))

        self.weights = pd.DataFrame(weights,columns=["weights"],index=self.getFundNames(self.funds))
        self.TabuSet = []
        #self.findInitialBest()
    
    def Evaluate(self,weights):
        asset = .7*(self.evaluateAssetOverlap(weights))
        discount = .3*self.evaluateDiscount(weights)
        return asset+discount
    
    def changeWeights(self,fund1,fund2,weights,change):
        change = min(self.getFundWeight(fund2,weights),change)
        weights.loc[fund1,"weights"] += change
        weights.loc[fund2,"weights"] -= change
        return weights
    
    def EvaluateChange(self,fund1,fund2,weights,change):
        weights = weights.copy()
        weights = self.changeWeights(fund1,fund2,weights,change)
        return self.Evaluate(weights)
    
    def getFundWeight(self,fund,weights):
        return weights.loc[fund][0]
    def findIdealChange(self,fund1,fund2,weights,change):
        control = self.Evaluate(weights)

        while self.EvaluateChange(fund1,fund2,weights,change)>control:
            #print(self.getFundWeight(fund2,weights), self.EvaluateChange(fund1,fund2,weights,change), control)
            self.changeWeights(fund1,fund2,weights,change)
            control = self.Evaluate(weights)
           
            
                
        return control, weights
    
    def makeAdjustment(self,fund1,fund2,weights,change):
        control = self.Evaluate(weights)
        
        up = self.EvaluateChange(fund1,fund2,weights,change)
        
        down = self.EvaluateChange(fund2,fund1,weights,change)
        
        if down>control and down>up:
            fund1, fund2 = fund2, fund1
        elif control>=up: #if down>control but smaller than up that wouldn't make sense so...
            print(up,down,control)
            return control,weights
        
        self.changeWeights(fund1,fund2,weights,change)
        print("HERE", fund2, fund2, self.getFundWeight(fund2,weights))
        value, weights = self.findIdealChange(fund1,fund2,weights,change)
        
        return value, weights
    
    def randomFunds(self):
        fund2 = random.choice(self.weights.index)
        fund1 = random.choice(self.weights.index)
        
        if self.getFundWeight(fund1,self.weights)==0 and self.getFundWeight(fund2,self.weights)==0: 
            return self.randomFunds()
        return fund1, fund2
    
    def getOption(self,fund1,fund2):
        change = .1
        value, weights = self.makeAdjustment(fund1, fund2, self.weights, .01)
        return Adjustment(fund1=fund1,fund2=fund2,value=value,weights=weights)
    
            
    def runIteration(self):
        options = [self.randomFunds() for x in range(self.numOptions)]
        print(options)
        print(list(map(lambda funds: [self.getFundWeight(funds[0],self.weights),self.getFundWeight(funds[1],self.weights)], options)))
        options = list(filter(lambda option: not self.inTabuSet(option),options))
        
        options = [self.getOption(fund1,fund2) for fund1,fund2 in options]
        
        print([option.getValue() for option in options])
        bestIndx = np.argmax([option.getValue() for option in options])

        option = options[bestIndx]
        self.weights = option.getWeights()
        self.TabuSet.append(option)
    
    def printEvaluation(self):
        print("Simple Overlap %.2f" % self.simpleOverlap(self.weights))
        print("Add Stocks Overlap %.2f" % self.addStocksOverlap(self.weights))
        print("Weighted Discount %.2f" % self.getWeightedDiscount(self.weights))
        
    def runModel(self):
        self.printEvaluation()
        
        for x in range(100):
            self.runIteration()
            self.printEvaluation()
            print("\n\n")

In [165]:
obj = runModel()
obj.runModel()

Simple Overlap 32.80
Add Stocks Overlap 62.75
Weighted Discount -5.29
[('CHN', 'AWP'), ('AIO', 'DPG'), ('GLQ', 'DPG'), ('IGR', 'JEQ'), ('CII', 'BSTZ'), ('GLQ', 'CEM'), ('ETJ', 'CEM'), ('BME', 'AEF'), ('CET', 'MIE'), ('DPG', 'CTR')]
[[0.0008569426529696612, 0.024637762494424696], [0.002085846746502648, 0.03516307260012754], [0.01663819552433484, 0.03516307260012754], [0.004259266095090835, 0.019739995078893798], [0.014832281681756997, 0.02659626380449391], [0.01663819552433484, 0.03284873606911774], [0.030259356173141624, 0.03284873606911774], [0.011855793577177044, 0.021754720239325724], [0.009184295438473399, 0.03603010247347855], [0.03516307260012754, 0.004977266932430434]]
HERE CHN CHN 0.0
HERE DPG DPG 0.02516307260012754
HERE GLQ GLQ 0.006638195524334841
HERE JEQ JEQ 0.009739995078893797
HERE BSTZ BSTZ 0.01659626380449391
31.979320752734647 31.9982727946944 31.9982727946944
HERE CEM CEM 0.02284873606911774
HERE AEF AEF 0.011754720239325724
HERE MIE MIE 0.026030102473478547
HERE CTR

45.083647662065616 44.8245230365147 45.083647662065616
HERE CTR CTR 0.0
45.063679954008606 45.180368431516456 45.180368431516456
HERE BME BME 0.013973235259758675
45.16117437731796 45.61245407231651 45.61245407231651
HERE BOE BOE 0.04187198901117004
HERE BGR BGR 0.004005238780264351
45.34634452603473 45.70122901548045 45.70122901548045
45.372050977571675 45.70122901548045 45.70122901548045
45.408389826958626 45.70122901548045 45.70122901548045
[45.083647662065616, 45.180368431516456, 45.180368431516456, 45.61245407231651, 45.61245407231651, 45.644579417023344, 45.70122901548045, 45.70122901548045, 45.70122901548045, 45.70122901548045]
Simple Overlap 51.90
Add Stocks Overlap 71.29
Weighted Discount -7.79



[('IAF', 'IAF'), ('IAF', 'RFI'), ('UTF', 'CEM'), ('CET', 'CET'), ('GLQ', 'AGD'), ('BGR', 'CEN'), ('EXD', 'CEN'), ('RNP', 'RQI'), ('EOS', 'BGR'), ('GLQ', 'ETJ')]
[[0.04580774920475396, 0.04580774920475396], [0.04580774920475396, 0.0], [0.03230214567428541, 0.0], [0.10235921205117465, 

49.0387034756877 49.118772381831164 49.118772381831164
49.07096080569587 49.118772381831164 49.118772381831164
HERE BIF BIF 0.04
49.13467903957356 49.09377924132497 49.13467903957356
48.833971718455295 49.13467903957356 49.13467903957356
49.02425576185985 49.13467903957356 49.13467903957356
49.13467903957356 49.03805642421578 49.13467903957356
[49.118772381831164, 49.118772381831164, 49.118772381831164, 49.118772381831164, 49.118772381831164, 49.13467903957356, 49.13467903957356, 49.13467903957356, 49.13467903957356, 49.13467903957356]
Simple Overlap 55.12
Add Stocks Overlap 72.96
Weighted Discount -8.66



[('CII', 'CHN'), ('CEN', 'BGR'), ('BIF', 'RNP'), ('CII', 'EMO'), ('SZC', 'BSTZ'), ('ETJ', 'PEO'), ('EOI', 'DSE'), ('EOS', 'BOE'), ('EOI', 'AGD'), ('DPG', 'IAF')]
[[0.07219105361048607, 0.0], [0.013963386446209066, 0.01], [0.04, 0.0], [0.07219105361048607, 0.0], [0.005448121195345049, 0.0], [0.0, 0.006813162373225294], [0.08085125750116627, 0.0], [0.07711333102252371, 0.0418719890111

51.379262179049846 51.154101419454335 51.379262179049846
51.379262179049846 51.288120797558705 51.379262179049846
51.379262179049846 51.28134931829749 51.379262179049846
HERE BIF BIF 0.03
51.51056638035709 51.51056638035709 51.51056638035709
51.51056638035709 51.414503576175086 51.51056638035709
51.41469695285767 51.51056638035709 51.51056638035709
51.51056638035709 51.44750076598795 51.51056638035709
[51.379262179049846, 51.379262179049846, 51.379262179049846, 51.379262179049846, 51.379262179049846, 51.51056638035709, 51.51056638035709, 51.51056638035709, 51.51056638035709, 51.51056638035709]
Simple Overlap 59.40
Add Stocks Overlap 75.32
Weighted Discount -8.39



[('BDJ', 'CHN'), ('AOD', 'AIO'), ('IAF', 'CET'), ('AIO', 'AOD'), ('CTR', 'AWP'), ('SZC', 'EXD'), ('JEQ', 'ADX'), ('DSE', 'CEN'), ('BDJ', 'ASA'), ('EMO', 'SRV')]
[[0.08843583762437937, 0.0], [0.013375219434529408, 0.0], [0.0, 0.14728144222253167], [0.0, 0.013375219434529408], [0.0, 0.005494705147394356], [0.005448121195345049

HERE CPZ CPZ 0.013010716487371088
53.938195540090724 53.668020480865266 53.938195540090724
53.938195540090724 53.83954572483904 53.938195540090724
53.8190943921144 53.938195540090724 53.938195540090724
53.938195540090724 53.818892833663455 53.938195540090724
53.938195540090724 53.59667292318421 53.938195540090724
53.938195540090724 53.83687787563324 53.938195540090724
HERE EOI EOI 0.11085125750116626
54.30108278926342 54.42572027984035 54.42572027984035
[53.812825760580814, 53.938195540090724, 53.938195540090724, 53.938195540090724, 53.938195540090724, 53.938195540090724, 53.938195540090724, 53.938195540090724, 54.42572027984035, 54.42572027984035]
Simple Overlap 62.58
Add Stocks Overlap 77.17
Weighted Discount -9.37



[('CTR', 'EOI'), ('RQI', 'DPG'), ('BDJ', 'DNP'), ('BOE', 'MIE'), ('AOD', 'RQI'), ('RQI', 'MIE'), ('ADX', 'STK'), ('CTR', 'ADX'), ('GRF', 'AIO'), ('ADX', 'ADX')]
[[0.0, 0.060851257501166274], [0.01, 0.0], [0.02301071648737109, 0.0], [0.03187198901117004, 0.0], [0.0133752

...

55.67019111889189 55.752620294770026 55.76029512945835
55.54688183927997 55.76029512945835 55.76029512945835
55.52397803485834 55.76029512945835 55.76029512945835
55.76029512945835 55.6149997111323 55.76029512945835
[55.76029512945835, 55.76029512945835, 55.76029512945835, 55.76029512945835, 55.76029512945835, 55.76029512945835, 55.76029512945835, 55.76029512945835, 55.76029512945835, 55.76029512945835]
Simple Overlap 62.90
Add Stocks Overlap 77.36
Weighted Discount -10.17



[('CPZ', 'EOS'), ('BIF', 'CLM'), ('BGR', 'BOE'), ('GRF', 'AGD'), ('CEM', 'BOE'), ('GRF', 'CEM'), ('AIO', 'BOE'), ('SZC', 'SZC'), ('BDJ', 'IAF'), ('AGD', 'CII')]
[[0.0, 0.07509711167082568], [0.005494705147394353, 0.0], [0.0, 0.02187198901117004], [0.05415351894085737, 0.0], [0.0, 0.02187198901117004], [0.05415351894085737, 0.0], [0.0, 0.02187198901117004], [0.02, 0.02], [0.03301071648737109, 0.0], [0.0, 0.042191053610486064]]
55.708340646822194 55.76029512945835 55.76029512945835
55.76029512945835 55.6393522677590

55.87550266697808 55.95574044269861 55.95574044269861
55.90892696825841 55.95571676607935 55.95574044269861
55.95574044269861 55.70897477404251 55.95574044269861
55.88052544758672 55.95574044269861 55.95574044269861
55.85272529655198 55.95574044269861 55.95574044269861
55.72085779251357 55.95574044269861 55.95574044269861
55.79688000811083 55.95574044269861 55.95574044269861
55.882785791273854 55.95574044269861 55.95574044269861
[55.95574044269861, 55.95574044269861, 55.95574044269861, 55.95574044269861, 55.95574044269861, 55.95574044269861, 55.95574044269861, 55.95574044269861, 55.95574044269861, 55.95574044269861]
Simple Overlap 62.96
Add Stocks Overlap 77.39
Weighted Discount -10.40



[('IAF', 'BOE'), ('BOE', 'RQI'), ('EOS', 'CLM'), ('EMO', 'BDJ'), ('CEN', 'DNP'), ('EXD', 'EOS'), ('ADX', 'RQI'), ('AOD', 'CEN'), ('ADX', 'EXD'), ('AOD', 'PEO')]
[[0.0, 0.011871989011170038], [0.011871989011170038, 0.0], [0.03509711167082568, 0.0], [0.0, 0.03301071648737109], [0.02945809159360342, 0.0]

55.98036625018964 56.00260931978929 56.00260931978929
55.63455216282538 56.00260931978929 56.00260931978929
55.93248009472456 56.00260931978929 56.00260931978929
56.00260931978929 55.93226058112436 56.00260931978929
55.8973445699836 56.00260931978929 56.00260931978929
[55.992163237863544, 55.992163237863544, 56.00260931978929, 56.00260931978929, 56.00260931978929, 56.00260931978929, 56.00260931978929, 56.00260931978929, 56.00260931978929, 56.00260931978929]
Simple Overlap 63.23
Add Stocks Overlap 77.55
Weighted Discount -10.32



[('GRF', 'IAF'), ('RNP', 'GRF'), ('BOE', 'BUI'), ('BDJ', 'BMEZ'), ('GRF', 'CRF'), ('IAF', 'GRF'), ('RNP', 'PEO'), ('NFJ', 'CET'), ('AGD', 'CET'), ('PEO', 'BDJ')]
[[0.04415351894085737, 0.0], [0.0, 0.04415351894085737], [0.011871989011170038, 0.0], [0.04301071648737109, 0.0], [0.04415351894085737, 0.0], [0.0, 0.04415351894085737], [0.0, 0.04262091157797924], [0.0, 0.23065666165706114], [0.0, 0.23065666165706114], [0.04262091157797924, 0.04301071648737109]]
56.0

56.02578204094236 55.829683293493325 56.02578204094236
55.96262911804607 56.02578204094236 56.02578204094236
56.02578204094236 55.72925088094992 56.02578204094236
55.834052454837945 56.02578204094236 56.02578204094236
56.02578204094236 55.93159886617615 56.02578204094236
HERE GRF GRF 0.04415351894085737
56.03811580216133 55.92325992183075 56.03811580216133
56.02329204122934 55.993429244186885 56.03811580216133
56.03811580216133 55.90095116975766 56.03811580216133
56.03811580216133 55.97287900669283 56.03811580216133
[56.02578204094236, 56.02578204094236, 56.02578204094236, 56.02578204094236, 56.02578204094236, 56.03811580216133, 56.03811580216133, 56.03811580216133, 56.03811580216133, 56.03811580216133]
Simple Overlap 63.74
Add Stocks Overlap 77.86
Weighted Discount -10.16



[('PEO', 'AIO'), ('CET', 'BSTZ'), ('AIO', 'CEN'), ('CET', 'EMO'), ('BDJ', 'NFJ'), ('CET', 'DPG'), ('BIF', 'PEO'), ('GLQ', 'BDJ'), ('IAF', 'EOI'), ('CET', 'RFI')]
[[0.04262091157797924, 0.0], [0.22065666165706113, 

HERE EOS EOS 0.04509711167082568
55.94906507967246 56.08006994456475 56.08006994456475
56.08006994456475 55.79667129620036 56.08006994456475
56.08006994456475 55.832500708509 56.08006994456475
HERE EOI EOI 0.09085125750116627
55.71631432967226 56.09167533408845 56.09167533408845
[56.06861982687305, 56.06861982687305, 56.07605672624126, 56.07605672624126, 56.08006994456475, 56.08006994456475, 56.08006994456475, 56.08006994456475, 56.09167533408845, 56.09167533408845]
Simple Overlap 63.81
Add Stocks Overlap 77.91
Weighted Discount -10.29



[('EMO', 'EOS'), ('CII', 'AEF'), ('BDJ', 'CHN'), ('GRF', 'RNP'), ('CET', 'AWP'), ('EOI', 'BSTZ'), ('ADX', 'CII'), ('BGR', 'SZC'), ('ADX', 'IAF'), ('BGR', 'ADX')]
[[0.0, 0.04509711167082568], [0.02219105361048606, 0.0], [0.04301071648737109, 0.0], [0.04415351894085737, 0.0], [0.22065666165706113, 0.0], [0.09085125750116627, 0.0], [0.46008868794948005, 0.02219105361048606], [0.0, 0.01], [0.46008868794948005, 0.0], [0.0, 0.46008868794948005]]
55.94969627

In [166]:
obj.weights.sort_values(by="weights",ascending=False)

,weights
ADX,0.460089
CET,0.220657
EOI,0.090851
EOS,0.045097
GRF,0.044154
BDJ,0.043011
PEO,0.042621
CII,0.022191
BOE,0.021330
SZC,0.010000


In [147]:
Discount().discount

,Effective,Distribution,Discount,52W Discount,longterm,value
Ticker,,,,,,
ADX,0.00,20.55,-13.27,-13.74,-4.304029,36.468780
AEF,7.90,2.64,-12.89,-14.09,-10.989011,33.155048
AGD,6.94,7.51,-12.70,-13.11,-3.754579,35.023824
AIO,2.95,5.05,-6.89,-10.25,-30.769231,9.423681
AOD,0.09,7.84,-13.81,-13.28,4.853480,41.063699
ASA,0.00,0.09,-15.96,-15.45,4.670330,47.143752
AWP,7.68,9.28,-12.22,-13.72,-13.736264,30.325558
BCX,1.20,6.63,-14.22,-15.29,-9.798535,37.350788
BDJ,0.27,7.19,-9.53,-9.34,1.739927,27.800770


In [114]:
def t(ab):
    ab = ab[:]
   
    ab.iloc[0] = [1,3]
    print(ab)
    
    
ki = pd.DataFrame([[1,2]])
t(ki)
ki
#i set the list copy to that and then though I could efdit it after word. but psych apparently that was me editing it and seeting list to something else doesn;t work othersie

   0  1
0  1  3


,0,1
0,1,3
